# Recurrent(Feedback) Neural Network

In [ ]:
- for sequential data processing

this is a sweet apple


- nlp tasks
- speech recognition
- time-series prediction



In [ ]:
- memorize parts of the inputs and use them to make accurate predictions

## Feed Forward Network


<img src='r1.png' />

# RNN

<img src='r2.png' />

In [ ]:
                 this is a red ....{apple}
0 - a            [0
1 - are           0
2 - apple         0
3                 0
.                 0
4100 - this       1       
.                 0
10000             0]

encoding -> one-hot encoding

word embeddings -> word2vec - GloVe


steps in RNN

1 - Input a dataset
2 - complex computation - randomly initializing variables(weights and biases)

y = f(x) = Wx + b

3 - generate prediction
4 - compare the prediction with expected values and it will give us an error
5 - propogate the error back to the same path in the network and adjust the variables

6 - repeat 1 - 5 until we have optimum variables

7 - prediction is made on unseen data


# Formula

<img src='r3.png' />

In [ ]:
1 - holds the information about the precious words in the sequence
h0 will initialy be zero

f() -> activation function -> tanh or sigmoid

2 - calculate the predicted word vector, we use softmax to produce vector (probability distribution)
summing up to 1

3 - cross entropy - loss function

# problems with RNN

In [ ]:
1 - Exploding gradients
- large weights updates

2 - Vanishing Gradients
- gradient will goes to zero


cause
1 - chain in multiplication
2 - long sequences 

# RNN Architectures

In [ ]:
1 - Bidirectional RNN  (BRNN)
2 - Gated Recurrent Unit (GRU)
3 - Long Short Term Memory(LSTM)

# Implementation using PyTorch

In [2]:
import torch
import torch.nn as nn

import torch.optim as optim
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
# define RNN

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        
        out, _ = self.rnn(x)
        
        out = self.fc(out[:, -1, :])
        
        return out

In [15]:
# hyperparameters

input_size = 28
hidden_size = 128
output_size = 10   # number of classes (0-9 digits)

learning_rate = 0.001
batch_size = 64

num_epochs = 5

In [6]:
# load the dataset

transform = transforms.Compose([transforms.ToTensor()])

train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = MNIST(root='./data', train=False, transform=transform)


100%|██████████| 9912422/9912422 [00:13<00:00, 758806.55it/s] 


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 181698.82it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 2364234.34it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [7]:
# create the data loaders

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [16]:
# model, loss function, optimizer

model = SimpleRNN(input_size, hidden_size, output_size).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
# training

for epoch in range(num_epochs):
    for batch, (images, labels) in enumerate(train_loader):
        
        images, labels = images.view(-1, 28, 28).to(device), labels.to(device)
        
        # forward pass
        outputs = model(images)
        
        # loss calculation
        loss = criterion(outputs, labels)
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}'.format(epoch, num_epochs))

Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5


In [19]:
# Evaluation

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.view(-1, 28, 28).to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print(f'Accuracy on the test set: {100 * correct / total}%')

Accuracy on the test set: 93.87%
